# Presentation Climate Modelling

In [306]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
import hvplot.pandas as hv
import xarray as xr
import hvplot.xarray
import cartopy.crs as ccrs
from scipy.interpolate import griddata
from scipy import interpolate
import geopandas as gpd
import colorcet as cc
from colorcet.plotting import swatch, swatches
from operator import itemgetter 

In [307]:
os.chdir('D:\\OneDrive\\GroundwatCh\\TUD\\Climate\\Modelling\\EdGCM\\Output/Doubled_CO2_2/diagnostics_Doubled_CO2_2/plots_Doubled_CO2_2/Doubled_CO2_2_2005-2100')
files_lst = [file for file in os.listdir()]
var_names_lst = [file.split('_')[-1].split('.')[0] for file in files_lst]

In [308]:
files_lst = ['Doubled_CO2_2_EVAPOR.xls','Doubled_CO2_2_PRECIP.xls', 'Doubled_CO2_2_SRFAIRTMP.xls']
var_names_dct = {'Evaporation' : 'mm/day',
                 'Precipitation': 'mm/day',
                 'Surface air temperature' : '°C'}

In [309]:
def DataFrame (index):
    #trying one file
    fn_i = index
    fn = files_lst[index]
    df = pd.read_csv(fn, sep = '\t', header = 1).rename(columns = {' Years' : 'Years'})
    df['category'] = index
   
    return df

In [310]:
Etp_df = DataFrame(0)
P_df = DataFrame(1)
Sat_df = DataFrame(2)

In [311]:
df = pd.concat([Etp_df, P_df, Sat_df])
df.loc [ df.category == 0, 'category'] = 'Evapotranspiration mm/day'
df.loc [ df.category == 1, 'category'] = 'Precipitation mm/day'
df.loc [ df.category == 2, 'category'] = 'Surface Air Temperature °C'

In [312]:
text1 = pn.pane.Markdown ('''
# <center>Simulations for twice the concentration of carbon dioxide by 2100</center>
<br>
<center>*Presentation by: Saulo Vieira da Silva Filho*</center>
''',
                          align = 'center', style = {'font-size' : '1.5em'})
r0 =   pn.layout.VSpacer()
c0  =  pn.layout.HSpacer(width = 100)
r1 = pn.Column(c0,text1, align = 'center')

col = pn.Row(r1, align = 'center', margin = (200, 300, 200, 300))
tabs = pn.Tabs(('Title', col))

In [313]:
pn.config.sizing_mode="stretch_width"
width = 1400


text = pn.pane.Markdown('''
## Time series - average surface temperature
<br>
**1.** <ins> Slight increase of all variables </ins>.  Overall, the variables increase and stabilize together, and a <ins> logical order is 
observed </ins> in which the open <ins> ocean has the greatest absolute values, followed by the ocean. Land has intermediate values and oceanic ice, 
the lowest. </ins>
<br>
**2.** The breakthrough curves <ins> flatten off in around 2060 </ins>
<br>
**3.** <ins> Lower average values </ins> in continental areas
<br>
**4.** Neligible decrease of evapotranspiration from oceanic ice sheets
<br>
**5.** Precipitation has stronger oscillation that the two other variables previously presented
    - Stronger oscillation of Precipitation in oceanic ice sheets

''', width = width ,
                        margin = (10,10,30,10),
                        style={'font-size': '1.2em'}
)


########### SAT
df_ = df [df.category.str.startswith('Surf') ]
Sat_plot = df_.hvplot.line(x = 'Years', y = df_.columns[1:-1],
                           ylabel = df_.category.unique()[0],
                           title = 'Time series for twice the [CO2] of pre-industrial levels (1958) ',
                            height = 500, shared_axes=False)

r1 = pn.Row (text, width = width)
r2 = pn.Row (Sat_plot, width = width,  align = 'center')
Sat_col = pn.Column(r1,r2,align = 'center', )



########### evapotranspiration

text = pn.pane.Markdown('''
## Time series - average evapotranspiration
<br>
**1.** <ins> Slight increase of all variables </ins>.  Overall, the variables increase and stabilize together, and a <ins> logical order is 
observed </ins> in which the open <ins> ocean has the greatest absolute values, followed by the ocean. Land has intermediate values and oceanic ice, 
the lowest. </ins>
<br>
**2.** The breakthrough curves <ins> flatten off in around 2060 </ins>
<br>
**3.** <ins> Lower average values </ins> in continental areas
<br>
**4.** **Neligible decrease of evapotranspiration from oceanic ice sheets **
    - It might be related to its decrease in volume 
<br>
**5.** Precipitation has stronger oscillation that the two others encompassed by this presentation
    - Stronger oscillation of Precipitation in oceanic ice sheets


''', width = width ,
                        margin = (10,10,30,10),
                        style={'font-size': '1.2em'}
)

df_ = df [ df.category.str.startswith('Ev') ]
Etp_plot = df_.hvplot.line(x = 'Years', y = df_.columns[1:-1],
                           ylabel = df_.category.unique()[0],
                           title = 'Time series for twice the [CO2] of pre-industrial levels (1958) ',
                            height = 500,  ylim= (0,5), shared_axes=False)     
r1 = pn.Row (text, width = width)
r2 = pn.Row (Etp_plot, width_policy = "auto")
Etp_col = pn.Column(r1,r2, align = 'center')



########### prectipitation

text = pn.pane.Markdown('''
## Time series - average precipitation
<br>
**1.** <ins> Slight increase of all variables </ins>.  Overall, the variables increase and stabilize together, and a <ins> logical order is 
observed </ins> in which the open <ins> ocean has the greatest absolute values, followed by the ocean. Land has intermediate values and oceanic ice, 
the lowest. </ins>
<br>
**2.** The breakthrough curves <ins> flatten off in around 2060 </ins>
<br>
**3.** <ins> Lower average values </ins> in continental areas
<br>
**4.** Neligible decrease of evapotranspiration from oceanic ice sheets
    - It might be related to its decrease in volume 
<br>
**5.** **Precipitation has stronger oscillation** that the two others encompassed by this presentation
    - Stronger oscillation of Precipitation in oceanic ice sheets


''', width = width ,
                        margin = (10,10,30,10),
                        style={'font-size': '1.2em'}
)

df_ = df [ df.category.str.startswith('Pr') ]
P_plot = df_.hvplot.line(x = 'Years', y = df_.columns[1:-1],
                           ylabel = df_.category.unique()[0],
                           title = 'Time series for twice the [CO2] of pre-industrial levels (1958) ',
                            height = 500, ylim= (0,5), shared_axes=False)            

r1 = pn.Row (text, width = width)
r2 = pn.Row (P_plot, width = width,  align = 'center')
P_col = pn.Column(r1,r2, align = 'center',)


########### append


tabs.append(('TS - SAT', Sat_col))
tabs.append(('TS - Etp', Etp_col))
tabs.append(('TS - P', P_col))

## Comparison of decadal output


1. first and last decade for trend simulation (linear, exponential, IPCC6)
2. Last decades of equilibrium simulations and controlrun2005 (data available)

In [314]:
os.chdir('d:\\OneDrive\\GroundwatCh\\TUD\Climate\\Modelling\\EdGCM\\Output\\Doubled_CO2_2\\diagnostics_Doubled_CO2_2\\maps_Doubled_CO2_2')
os.listdir()

['Control2005.2001-2010ij.nc',
 'Control2005.2091-2100ij.nc',
 'Doubled_CO2_2.2005-2020ij.nc',
 'Doubled_CO2_2.2091-2100ij.nc']

### 2CO2 simulation 2005-2020

In [315]:
pn.config.sizing_mode="stretch_width"

#OPENING 2010-2020
ds = xr.open_dataset('Doubled_CO2_2.2005-2020ij.nc')
months_lst = list(ds.months.to_numpy())

etp_cscale = (0, 10)
p_cscale = (0,14)
t_cscale = (-50,40)


#params
width = 700; height = int(width / 2.2)

#frame
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")   )

#etp
da = ds.Evap.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet',
                          title = '''Annual Evapotranspiration [{}]: 1st decade'''.format(da.attrs["units"]),
                            clim = etp_cscale)
yr_etp_plt = (nc_plot * word_plt )


#P
da = ds.Precip.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet_r',
                          title = 'Annual Precipitation [{}]: 1st decade'.format(da.attrs['units']),
                         clim = p_cscale)
yr_p_plt = (nc_plot * word_plt )


#SAT
da = ds.SurfAirTemp.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', 
                          title = 'Annual Surface air Temperature [{}]: 1st decade'.format(da.attrs['units']),
                         clim = t_cscale)
yr_sat_plt = (nc_plot * word_plt )





text = pn.pane.Markdown('''
### General characteristics 
* <ins> Wetter </ins> regions evapotranspirate more. <ins> Wettest </ins> area is Moonson Asia.
<br>
* <ins> Lowest </ins> temperature in Antartica
<br>
### Constraints (South America)
* The <ins> coarse resolution </ins> of the model output can be <ins> misleading </ins> if used at a country scale.
<br>
* In the case of <ins>South America</ins>, the continent chosen in the first presentation, we observe <ins> non-expected patterns</ins> 
in the pixels that sorround the Amazon rain-forest.
    * Precipitation in the biome Caatinga should be lower than <ins> 8 mm/day </ins>.
    <br>
    * The <ins> Amazon rain-forest should not have bigger evapotranspiration than Caatinga. </ins>
''', width = width , style={'font-size': '1.2em'}, margin = (30,30,10,30)
)



title = pn.pane.Markdown('# Maps - Twice [CO2] for the 1st decade')
row1 = pn.Row(yr_etp_plt, yr_p_plt)
row2 = pn.Row(yr_sat_plt,text)
col_sim_20052020 = pn.Column (title,row1, row2, margin = (0,0,0,0))

tabs.append(('Map_abs_1dec', col_sim_20052020))

### 2CO2  2091-2100

In [316]:
#OPENING 2010-2020
ds = xr.open_dataset('Doubled_CO2_2.2091-2100ij.nc')
months_lst = list(ds.months.to_numpy())

#params
width = 700; height = int(width / 2.2)

#frame
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")   )

#etp
da = ds.Evap.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet',
                          title = '''Annual Evapotranspiration [{}]: last decade'''.format(da.attrs["units"]),
                         clim = etp_cscale)
yr_etp_plt = (nc_plot * word_plt )


#P
da = ds.Precip.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet_r',
                          title = 'Annual Precipitation [{}]: last decade'.format(da.attrs['units']),
                         clim = p_cscale)
yr_p_plt = (nc_plot * word_plt )


#SAT
da = ds.SurfAirTemp.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', 
                          title = 'Annual Surface air Temperature [{}]: last decade'.format(da.attrs['units']),
                         clim = t_cscale)
yr_sat_plt = (nc_plot * word_plt )



text = pn.pane.Markdown('''
### General characteristics
* <ins>Same patterns</ins> are observed in the last decade.
<br>
<br>
* Temperature and evapotranspiration <ins>increase.</ins>
<br>
<br>
* <ins>Precipitation increases in ocean and coastal areas<ins>.

''', width = width , style={'font-size': '1.2em'}, margin = (30,30,10,30)
)




title = pn.pane.Markdown('# Maps - Twice [CO2] for the last decade')
row1 = pn.Row(yr_etp_plt, yr_p_plt)
row2 = pn.Row( yr_sat_plt, text)
col_sim_20912100 = pn.Column (title,row1, row2, margin = (0,0,0,0))

tabs.append(('Map_abs_2dec', col_sim_20912100))

### 2CO2 - Comparison - first an last decade for trend simulation (linear, exponential, IPCC6)

In [317]:
## creating colormap
neg = cc.coolwarm[:128]
pos = cc.coolwarm[128:]
index = np.linspace(0,len(neg)-1,40).astype(int)
neg = itemgetter(*index)(neg)

custom_cmap = list(neg) + pos

custom = swatch( name = 'custom', cmap = custom_cmap)

In [318]:
fn2005 = 'Doubled_CO2_2.2005-2020ij.nc'
fn2091 =  'Doubled_CO2_2.2091-2100ij.nc'
ds2005 = xr.open_dataset(fn2005)
ds2091 = xr.open_dataset(fn2091)

#difference in the decade
ds = ds2091 - ds2005 #delta variables
ds.attrs = ds2005.attrs
months_lst = list(ds.months.to_numpy())
ds = ds.sel(months = months_lst[-1]) #get only annual values

d_etp_clim = (-2,2)
d_p_clim = (-3,3)
d_t_clim = (0,10)

#params
width = 700; height = int(width / 2.2)

#frame
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")   )

#etp
da = ds.Evap
da.attrs = ds2005.Evap.attrs
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = cc.coolwarm,
                          title = '''Annual Evapotranspiration [{}] '''.format(da.attrs["units"]),
                            clim = d_etp_clim)
yr_etp_plt = (nc_plot * word_plt )


#P
da = ds.Precip
da.attrs = ds2005.Precip.attrs
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = cc.coolwarm[::-1],
                          title = 'Annual Precipitation [{}]'.format(da.attrs['units']),
                             clim = d_p_clim)
yr_p_plt = (nc_plot * word_plt )


#SAT
da = ds.SurfAirTemp
da.attrs = ds2005.SurfAirTemp.attrs
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', 
                          title = 'Annual Surface air Temperature [{}]'.format(da.attrs['units']),
                             clim = d_t_clim)
yr_sat_plt = (nc_plot * word_plt )


text = pn.pane.Markdown('''
<br>
**1.** Rainier regions evapotranspirate more
<br>
<br>
**2.** Temperature and evapotranspiration increase
<br>
<br>
**3.** Precipitation increases in some portions: ocean and coastal areas
<br>
<br>
**4.** Highest increase of temperature is in the southern hemisphere - **Antarctic Peninsula can reach almost 10 degrees** difference on yearly averages
<br>
<br>
**5.** Temperatures differences in South America range between 2 and 4 celsius degrees

''', width = width , style={'font-size': '1.2em'}, margin = (10,30,10,30)
)


title = pn.pane.Markdown('# Maps - Scenario Simulations - first and last decade for the 2[CO2] scenario')
row1 = pn.Row(yr_etp_plt, yr_p_plt)
row2 = pn.Row(yr_sat_plt, text)
col_delta_sim = pn.Column (title,row1, row2, margin = (0,0,0,0)) #delta simulation

tabs.append(('Map_d_sim', col_delta_sim))

### 2CO2 - Comparison - Last decades of equilibrium simulations and controlrun2005 (data available)

In [319]:
## creating colormap
neg = cc.coolwarm[:128]
pos = cc.coolwarm[128:]
index = np.linspace(0,len(neg)-1,40).astype(int)
neg = itemgetter(*index)(neg)

custom_cmap = list(neg) + pos

custom = swatch( name = 'custom', cmap = custom_cmap)

In [320]:
fncontrol = 'Control2005.2091-2100ij.nc'
fnsim =  'Doubled_CO2_2.2091-2100ij.nc'
dsc = xr.open_dataset(fncontrol)
dss = xr.open_dataset(fnsim)
#difference in the decade

ds = dss - dsc #delta variables
months_lst = list(dsc.months.to_numpy())
ds = ds.sel(months = months_lst[-1] )

#params
width = 700; height = int(width / 2.2)

#frame
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")   )

#etp
da = ds.Evap
da.attrs = dss.Evap.attrs
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = cc.coolwarm,
                          title = '''Annual Evapotranspiration [{}] '''.format(da.attrs["units"]),
                            clim = d_etp_clim)
yr_etp_plt = (nc_plot * word_plt )


#P
da = ds.Precip
da.attrs = dss.Precip.attrs
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = cc.coolwarm[::-1],
                          title = 'Annual Precipitation [{}]'.format(da.attrs['units']),
                             clim = d_p_clim)
yr_p_plt = (nc_plot * word_plt )


#SAT
da = ds.SurfAirTemp
da.attrs = dss.SurfAirTemp.attrs
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', 
                          title = 'Annual Surface air Temperature [{}]'.format(da.attrs['units']),
                            clim = d_t_clim)
yr_sat_plt = (nc_plot * word_plt )



text = pn.pane.Markdown('''
<br>
**1.** Differences are consistent with the <ins> decadal differences, but are overall bigger</ins>.
<br>
<br>
**2.** Similar patterns for precipitation and evapotranspiration.
<br>
<br>
**3.** Highest difference for temperature in the <ins> Antarctic Peninsula </ins>. That shows that the doubling the pre-industrial concentration of CO2 can increase 
the temperature in until 10 deegres in comparison to the control simulation.
<br>
<br>
**4.** Considerable difference in the <ins>Arctic</ins>. 
''', width = width , style={'font-size': '1.2em'}, margin = (30,30,10,30)
)




title = pn.pane.Markdown('# Maps - differences between the last decades for the 2[CO2] scenario and control simulation')
row1 = pn.Row(yr_etp_plt, yr_p_plt)
row2 = pn.Row(yr_sat_plt, text)
col_simcon = pn.Column (title,row1, row2, margin = (0,0,0,0))

tabs.append (('Map_d_simxcom', col_simcon))

## Diagnostics

### By latitude

In [321]:
os.chdir('d:\\OneDrive\\GroundwatCh\\TUD\\Climate\\Modelling\\EdGCM\\Output\\Doubled_CO2_2\\diagnostics_Doubled_CO2_2\\tables_Doubled_CO2_2')
filesreg = [file for file in os.listdir() if file.startswith('ANN') and file.endswith('g.xls')]
files = [file for file in os.listdir() if file.startswith('ANN') and file.endswith('2.xls')]
files

['ANN2005-2020.avgDoubled_CO2_2.xls', 'ANN2091-2100.avgDoubled_CO2_2.xls']

In [322]:
df2005 = pd.read_csv(files[0], header = 6, sep ='\t',  na_values=['none'],  skipinitialspace = True)
cols = [i.replace(' ', '') for i in list(df2005.columns)]
df2005.columns = cols

df2005 = df2005 [['Latitude','PRECIP_(MM/DAY)', 'EVAPOR_(MM/DAY)', 'T_SURF_(.1_C)' , 'WATER_RUNOFF_Z0' , 'WATR_RUNOFF_MLD' ]]

units = [i.replace(' ', '') for i in list(df2005.iloc[0,:])]
df2005 = df2005.iloc[2:,:].reset_index(drop = True)

#preparing dataframes
global_df = df2005.iloc[0:24,:].reset_index(drop=True).astype('float64')
global_df['Label'] = 'Global'
land_df = df2005.iloc[28:28+24,:].reset_index(drop=True).astype('float64')
land_df['Label'] = 'Land'
ocean_df = df2005.iloc[56:56+24,:].reset_index(drop=True).astype('float64')
ocean_df['Label'] = 'Ocean'
oceanice_df = df2005.iloc[84:84+24].reset_index(drop=True).astype('float64')
oceanice_df['Label'] = 'Ocean Ice'
df = pd.concat([global_df, land_df, ocean_df, oceanice_df])
df['Period'] = '2005-2020'

In [323]:
df2005 = pd.read_csv(files[1], header = 6, sep ='\t',  na_values=['none'],  skipinitialspace = True)
cols = [i.replace(' ', '') for i in list(df2005.columns)]
df2005.columns = cols

df2005 = df2005 [['Latitude','PRECIP_(MM/DAY)', 'EVAPOR_(MM/DAY)', 'T_SURF_(.1_C)' , 'WATER_RUNOFF_Z0' , 'WATR_RUNOFF_MLD' ]]

units = [i.replace(' ', '') for i in list(df2005.iloc[0,:])]
df2005 = df2005.iloc[2:,:].reset_index(drop = True)

#preparing dataframes
global_df = df2005.iloc[0:24,:].reset_index(drop=True).astype('float64')
global_df['Label'] = 'Global'
land_df = df2005.iloc[28:28+24,:].reset_index(drop=True).astype('float64')
land_df['Label'] = 'Land'
ocean_df = df2005.iloc[56:56+24,:].reset_index(drop=True).astype('float64')
ocean_df['Label'] = 'Ocean'
oceanice_df = df2005.iloc[84:84+24].reset_index(drop=True).astype('float64')
oceanice_df['Label'] = 'Ocean Ice'
df_ = pd.concat([global_df, land_df, ocean_df, oceanice_df])
df_['Period'] = '2091-2100'

In [324]:
df = pd.concat([df, df_])
df['T_SURF_(.1_C)'] *= 0.1
df = df.rename(columns = {'T_SURF_(.1_C)' : 'T_SURF_(C)'})

In [325]:
# label_wid = pn.widgets.Select(name = 'Select', options = list(df.Label.unique()))

# def func(df, label_wid):
#     df = df [df.Label == label_wid]
#     return df

# funci = hvplot.bind(func, df, label_wid).interactive()

#### it doesn't work for html files

In [326]:
# plt1 = funci.hvplot.line(x = 'Latitude', y = df.columns[1:-1][0], by = 'Period'  )
# plt2 = funci.hvplot.line(x = 'Latitude', y = df.columns[1:-1][1], by = 'Period'  )
# plt3 = funci.hvplot.line(x = 'Latitude', y = df.columns[1:-1][2], by = 'Period'  )
# plt4 = funci.hvplot.line(x = 'Latitude', y = df.columns[1:-1][3], by = 'Period'  )

# plt = (plt1+ plt2 + plt3 + plt4).cols(2)

# r0 = pn.pane.Markdown('# Average annual values across the same longitude')
# r1 = pn.Row(pn.pane.Markdown('## Area '))
# r2 = pn.Row(plt, width = 1200)
# col_diag1 = pn.Column(r0, r1, r2)
# # col_diag1.show()

In [327]:
plt1 = df[df.Label == 'Global'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][0], by = 'Period' , shared_axes = False, height = 250)
plt2 = df[df.Label == 'Global'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][1], by = 'Period' , shared_axes = False, height = 250)
plt3 = df[df.Label == 'Global'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][2], by = 'Period' , shared_axes = False, height = 250)
plt4 = df[df.Label == 'Global'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][3], by = 'Period' , shared_axes = False, height = 250)

plt = (plt1+ plt2 + plt3 + plt4).cols(2)

r0 = pn.pane.Markdown('# Global average annual values across different latitudes')
t1 = '''
**1.** Smooth increase of <ins>surface temperature and evapotranspiration</ins> towards the equator.
<br>
**2.** <ins> Precipitation</ins> has a steep decrease in tropical latitudes (+- 30 degrees). That could be due to the fact that the wind currents flow 
from the tropics to the Equator (intertropical convergence zone).
<br>
**3.** <ins>High values of runoff</ins> at high latitudes. *The group is unsure about what this could be but wondered if this could be
representing the melting ice. Nevertheless, we expected the runoff to be bigger between the tropics.*
'''
r1 = pn.pane.Markdown( t1, style={'font-size': '1.2em'}, width = 1200)
r2 = pn.Row(plt, width = 1200, height = 500)
col_diag1a = pn.Column(r0, r1, r2)
# col_diag1a.show()
tabs.append (('Diagnostic A', col_diag1a))


C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this w

In [328]:
plt1 = df[df.Label == 'Ocean'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][0], by = 'Period'  ,shared_axes = False, height = 250)
plt2 = df[df.Label == 'Ocean'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][1], by = 'Period'  ,shared_axes = False, height = 250)
plt3 = df[df.Label == 'Ocean'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][2], by = 'Period'  ,shared_axes = False, height = 250)
plt4 = df[df.Label == 'Ocean'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][3], by = 'Period' ,shared_axes = False, height = 250) 

plt = (plt1+ plt2 + plt3 + plt4).cols(2)

r0 = pn.pane.Markdown('# Ocean average annual values across different latitudes')

t1 = '''
**1.** <ins> Similar patterns with the decadal with the global</ins>.
<br>
**2.** No <ins>runoff</ins> is observed in the ocean. 
<br>
**3.** Small decrease of <ins>evaporation</ins> in the Equator, despite the highest temperature.
'''

r1 = pn.pane.Markdown( t1, style={'font-size': '1.2em'}, width = 1200)
r2 = pn.Row(plt, width = 1200, height = 500)
col_diag1b = pn.Column(r0, r1, r2, height = 500)
# col_diag1b.show()

tabs.append (('Diagnostic B', col_diag1b))


C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this w

In [329]:
plt1 = df[df.Label == 'Land'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][0], by = 'Period' ,shared_axes = False, height = 250) 
plt2 = df[df.Label == 'Land'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][1], by = 'Period' ,shared_axes = False, height = 250) 
plt3 = df[df.Label == 'Land'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][2], by = 'Period' ,shared_axes = False, height = 250) 
plt4 = df[df.Label == 'Land'].hvplot.line(y = 'Latitude', x = df.columns[1:-1][3], by = 'Period' ,shared_axes = False, height = 250) 

plt = (plt1+ plt2 + plt3 + plt4).cols(2)

r0 = pn.pane.Markdown('# Land average annual values across the same longitude')
t1 = '''
**1.** <ins> Evapotranspiration in the land </ins> might be limited by the available water. 
The shape of the curve is similar to precipitation.
<br>
**2.** <ins>Unexpectedly</ins> high peak of runoff in high latitudes of the southern hemisphere.
'''
r1 = pn.pane.Markdown( t1, style={'font-size': '1.2em'}, width = 1200)
r2 = pn.Row(plt, width = 1200, height = 500)
col_diag1c = pn.Column(r0, r1, r2)
# col_diag1c.show()

tabs.append (('Diagnostic C', col_diag1c))

C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this w

### By Region

In [330]:
df = pd.read_csv(filesreg[0], header = 6, sep ='\t',  na_values=['none'],  skipinitialspace = True)
df.columns
cols = [i.replace(' ', '') for i in list(df.columns)]
df.columns = cols
df = df [['Region','PRECIP_(MM/DAY)', 'EVAPOR_(MM/DAY)', 'T_SURF_(.1_C)' , 'WATER_RUNOFF_Z0' , 'WATR_RUNOFF_MLD' ]]
units = [i.replace(' ', '') for i in list(df.iloc[0,:])]
df = df.iloc[2:,:].reset_index(drop = True)
cols = [i.replace(' ', '') for i in list(df.columns)]
df.columns = cols
df ['Period'] = '2005-2020'

In [331]:
df_ = pd.read_csv(filesreg[1], header = 6, sep ='\t',  na_values=['none'],  skipinitialspace = True)
cols = [i.replace(' ', '') for i in list(df_.columns)]
df_.columns = cols
df_ = df_ [['Region','PRECIP_(MM/DAY)', 'EVAPOR_(MM/DAY)', 'T_SURF_(.1_C)' , 'WATER_RUNOFF_Z0' , 'WATR_RUNOFF_MLD' ]]
units = [i.replace(' ', '') for i in list(df_.iloc[0,:])]
df_ = df_.iloc[2:,:].reset_index(drop = True)
cols = [i.replace(' ', '') for i in list(df_.columns)]
df_.columns = cols
df_ ['Period'] = '2091-2100'

In [332]:
df = pd.concat([df, df_])

In [333]:
for  col in df.columns[1:-1]:
    df[col] = df[col].astype('float64')
    
df['T_SURF_(.1_C)'] *= 0.1
df = df.rename(columns = {'T_SURF_(.1_C)' : 'T_SURF_(C)'})

In [334]:
table = df.groupby(['Region', 'Period'])['PRECIP_(MM/DAY)'].mean()
p_plt = table.hvplot.bar(rot = 90, xticks = 'T', width = 1500, height = 250,grid = True, xlabel ='')

table = df.groupby(['Region', 'Period'])['EVAPOR_(MM/DAY)'].mean()
etp_plt = table.hvplot.bar(rot = 90, xticks = 'T', width = 1500, height = 250,grid = True, xlabel ='')

table = df.groupby(['Region', 'Period'])[df.columns[3]].mean()
tsurf_plt = table.hvplot.bar(rot = 90, xticks = 'T', width = 1500, height =250, grid = True)


table = df.groupby(['Region', 'Period'])[df.columns[4]].mean()
roff1_plt = table.hvplot.bar(rot = 90, xticks = 'T', width = 1800, grid = True)

table = df.groupby(['Region', 'Period'])[df.columns[5]].mean()
roff2_plt = table.hvplot.bar(rot = 90, xticks = 'T', width = 1800, grid = True)

col_diag2 = pn.Column((p_plt + etp_plt + tsurf_plt).cols(1))

tabs.append (('Regional diagnostic', col_diag2))

In [335]:
text1 = pn.pane.Markdown ('''
# <center>Thank you</center> 
<br>
<br>
*<center>Saulo Vieira da Silva Filho 
<br>
Isadora Mendel Freccia
<br>
Joshua Antonio Fernando</center>*

''',
                          align = 'center', style = {'font-size' : '1.5em'})


r1 = pn.Row(height = 350)
r2 = pn.Row( text1)
vpage = pn.Column(r1,r2)

tabs.append (('Thank You', vpage))

tabs.show()

Launching server at http://localhost:60029


C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this w

In [338]:
os.chdir('d:\\OneDrive\\GroundwatCh\\TUD\\Climate\\Modelling\\EdGCM\\presentation')

In [336]:
tabs.save('presentation.html')

C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  data = [(k, group_type(v, **group_kwargs)) for k, v in
C:\ProgramData\Miniconda3\lib\site-packages\holoviews\core\data\pandas.py:222: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this w

# Old

# Control

In [97]:
#OPENING
ds = xr.open_dataset('Control2005.2001-2010ij.nc')
months_lst = list(ds.months.to_numpy())
#params
width = 700; height = int(width / 2)

#frame
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")   )

#etp
da = ds.Evap.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', title = 'Annual Evapotranspiration : 2001-2010')
yr_etp_plt = (nc_plot * word_plt )


#P
da = ds.Precip.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', title = 'Annual Precipitation : 2001-2010')
yr_p_plt = (nc_plot * word_plt )


#SAT
da = ds.SurfAirTemp.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', title = 'Annual Surface air  : 2001-2010')
yr_sat_plt = (nc_plot * word_plt )

title = pn.pane.Markdown('# Maps - Control Simulations 2001-2010')
row1 = pn.Row(yr_etp_plt, yr_p_plt)
row2 = pn.Row(pn.Spacer(width=int(width/2)), yr_sat_plt, pn.Spacer(width=int(width/2)))
col = pn.Column (title,row1, row2, margin = (0,0,0,0))
tabs.append (('Map 2001-2010', col))

### 2CO2 simulation 2010-2020

In [86]:
#OPENING 2010-2020
ds = xr.open_dataset('Doubled_CO2_2.2005-2020ij.nc')
months_lst = list(ds.months.to_numpy())

#params
width = 700; height = int(width / 2)

#frame
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")   )

#etp
da = ds.Evap.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', title = 'Annual Evapotranspiration : 2005-2020')
yr_etp_plt = (nc_plot * word_plt )


#P
da = ds.Precip.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', title = 'Annual Precipitation : 2005-2020')
yr_p_plt = (nc_plot * word_plt )


#SAT
da = ds.SurfAirTemp.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', title = 'Annual Surface air  : 2005-2020')
yr_sat_plt = (nc_plot * word_plt )


title = pn.pane.Markdown('# Maps - Control Simulations 2091-2100')
row1 = pn.Row(yr_etp_plt, yr_p_plt)
row2 = pn.Row(pn.Spacer(width=int(width/2)), yr_sat_plt, pn.Spacer(width=int(width/2)))
col = pn.Column (title,row1, row2, margin = (0,0,0,0))
tabs.append (('Map - 2CO2 - 2005-2020', col))

### 2CO2 simulation 2091-2100

In [93]:
#OPENING 2010-2020
ds = xr.open_dataset('Doubled_CO2_2.2091-2100ij.nc')
months_lst = list(ds.months.to_numpy())

#params
width = 700; height = int(width / 2)

#frame
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")   )

#etp
da = ds.Evap.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', title = 'Annual Evapotranspiration : 2091-2100')
yr_etp_plt = (nc_plot * word_plt )


#P
da = ds.Precip.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', title = 'Annual Precipitation : 2091-2100')
yr_p_plt = (nc_plot * word_plt )


#SAT
da = ds.SurfAirTemp.sel(months = months_lst[-1] )
word_plt = world.hvplot( width = width, height = height , color = None) 
nc_plot = da.hvplot.image(x = 'longitude', y = 'latitude',
                          width = width, height = height, cmap = 'jet', title = 'Annual Surface air  : 2091-2100')
yr_sat_plt = (nc_plot * word_plt )


title = pn.pane.Markdown('# Maps - Control Simulations 2091-2100')
row1 = pn.Row(yr_etp_plt, yr_p_plt)
row2 = pn.Row(pn.Spacer(width=int(width/2)), yr_sat_plt, pn.Spacer(width=int(width/2)))
col = pn.Column (title,row1, row2, margin = (0,0,0,0))
tabs.append (('Map - 2CO2 - 2091-2100', col))

# OLD

In [613]:
df = pd.DataFrame(da)
df.index = da.latitude
df.columns = da.longitude
df = df.sort_index(ascending = False)
df = df.reset_index().rename(columns = {'index' : 'latitude'})
df = pd.melt(df, id_vars = ['latitude'], var_name = 'longitude')
df.head()

,latitude,longitude,value
0,90.0,-175.0,0.009888
1,84.0,-175.0,0.020655
2,76.0,-175.0,0.024966
3,68.0,-175.0,0.073261
4,60.0,-175.0,1.902786


In [614]:
xx, yy = np.mgrid[da.longitude.to_numpy().min():da.longitude.to_numpy().max():da.longitude.size*10j,
                  da.latitude.to_numpy().min():da.latitude.to_numpy().max():da.latitude.size*10j]

z =  griddata(np.array(df[['latitude', 'longitude']]), 
              np.array(df['value']),
                       (xx, yy),
                       method='linear')

In [635]:
da =xr.Dataset(
    data_vars = dict (
        value = (['x','y'], z),
        ),
    coords = dict (
        Lat = (['x','y'], yy),
        Lon = (['x','y'], xx)
    )
).value

In [291]:
# fig = plt.figure(figsize = (25,10))
# ax = plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines() 
# da.plot()
# plt.show()